In [1]:
def english_to_predicate(sentence):
    sentence = sentence.rstrip('.?!')
    quantifier_keywords = ['all', 'every', 'each', 'some', 'a', 'an', 'no', 'none']
    sentence_lower = sentence.lower()
    quantifier = None
    for q in quantifier_keywords:
        if sentence_lower.startswith(q + ' '):
            quantifier = q
            break
    rest = sentence[len(quantifier):].strip()
    splitter = None
    if ' are ' in rest:
        splitter = ' are '
    elif ' is ' in rest:
        splitter = ' is '
    subject_part, predicate_part = rest.split(splitter, 1)
    negated = False
    if predicate_part.lower().startswith('not '):
        negated = True
        predicate_part = predicate_part[4:].strip()
    subject = subject_part.strip()
    predicate = predicate_part.strip()
    subject_singular = subject.rstrip('s') if subject.endswith('s') else subject
    predicate_singular = predicate.rstrip('s') if predicate.endswith('s') else predicate
    subject_pred = f"is_{subject_singular.lower()}"
    predicate_pred = f"is_{predicate_singular.lower()}"
    if negated:
        predicate_pred = f"¬{predicate_pred}"
    if quantifier in ['all', 'every', 'each']:
        return f"∀x ({subject_pred}(x) → {predicate_pred}(x))"
    elif quantifier in ['some', 'a', 'an']:
        return f"∃x ({subject_pred}(x) ∧ {predicate_pred}(x))"
    elif quantifier in ['no', 'none']:
        return f"∀x ({subject_pred}(x) → ¬{predicate_pred}(x))"


In [2]:

english_sentence = "All dinosaurs are extinct."
predicate_expr = english_to_predicate(english_sentence)
print(f"English to Predicate: {english_sentence} => {predicate_expr}")


English to Predicate: All dinosaurs are extinct. => ∀x (is_dinosaur(x) → is_extinct(x))


In [3]:

english_sentence = "Some animals are endangered."
predicate_expr = english_to_predicate(english_sentence)
print(f"English to Predicate: {english_sentence} => {predicate_expr}")


English to Predicate: Some animals are endangered. => ∃x (is_animal(x) ∧ is_endangered(x))


In [4]:

def predicate_to_english(expression):
    expr = expression.replace(' ', '')
    quantifier = None
    if expr.startswith('∀x'):
        quantifier = 'all'
        rest = expr[3:]
    elif expr.startswith('∃x'):
        quantifier = 'some'
        rest = expr[3:]
    elif expr.startswith('¬∃x'):
        quantifier = 'no'
        rest = expr[4:]
    inner = rest[1:-1]
    connective = None
    if '→' in inner:
        parts = inner.split('→', 1)
        connective = '→'
    elif '∧' in inner:
        parts = inner.split('∧', 1)
        connective = '∧'
    antecedent, consequent = parts[0].strip(), parts[1].strip()
    def parse_predicate(pred):
        neg = False
        if pred.startswith('¬'):
            neg = True
            pred = pred[1:]
        if pred.startswith('is_'):
            name = pred[3:].split('(x)')[0]
        else:
            name = pred.split('(x)')[0]
        return name, neg
    ant_name, ant_neg = parse_predicate(antecedent)
    cons_name, cons_neg = parse_predicate(consequent)
    ant_plural = ant_name + 's' if not ant_name.endswith('s') else ant_name
    if quantifier == 'all':
        if connective == '→':
            if cons_neg:
                return f"No {ant_plural} are {cons_name}."
            else:
                return f"All {ant_plural} are {cons_name}."
    elif quantifier == 'some':
        if connective == '∧' and not cons_neg:
            return f"Some {ant_plural} are {cons_name}."
    elif quantifier == 'no':
        if connective == '∧':
            return f"No {ant_plural} are {cons_name}."
    return "Unsupported expression structure"

In [5]:


predicate_expression = "∀x (is_dinosaur(x) → is_extinct(x))"
english_sentence = predicate_to_english(predicate_expression)
print(f"Predicate to English: {predicate_expression} => {english_sentence}")


Predicate to English: ∀x (is_dinosaur(x) → is_extinct(x)) => All s_dinosaurs are extinct.


In [6]:

predicate_expression = "∃x (is_animal(x) ∧ is_endangered(x))"
english_sentence = predicate_to_english(predicate_expression)
print(f"Predicate to English: {predicate_expression} => {english_sentence}")


Predicate to English: ∃x (is_animal(x) ∧ is_endangered(x)) => Some s_animals are endangered.
